# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list
for f in file_path_list:

# reading csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)

 # extracting each data row one by one and append it
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# uncomment the code below if you would like to get total number of rows
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS milaudacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('milaudacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Table Schema Design Explanations

In Apache Cassandra, we design tables based on the queries we need to run. The primary key structure is critical for performance. The primary key consists of a partition key (which determines data distribution) and optional clustering columns (which determine sort order within a partition).

### Table 1: `session_songs`

Purpose is to retrieve artist, song title, and song length for a specific session and item position.

**Primary Key:**
- **Partition Key:** `sessionId` - This is used as the partition key because Query 1 filters on `sessionId = 338`. By using `sessionId` as the partition key, all songs for a given session are stored together on the same node, enabling efficient retrieval.
- **Clustering Column:** `itemInSession` - This serves as a clustering column to ensure uniqueness (no two songs can have the same sessionId and itemInSession) and provides ordering within the partition, allowing us to efficiently retrieve a specific item in the session sequence.

**Why This Design:**
- Query 1 filters by both `sessionId` AND `itemInSession`, making `(sessionId, itemInSession)` an ideal composite primary key.
- Data is distributed by session across the cluster, and within each partition, rows are sorted by item position.

### Table 2: `user_session_songs`

Purpose is to retrieve artist, song title, and user name for a specific user and session, sorted by item in session.

**Primary Key:**
- **Partition Key:** `(userId, sessionId)` - This is a composite partition key because Query 2 filters on both `userId = 10` AND `sessionId = 182`. Using both columns as the partition key ensures that all songs for a specific user-session combination are co-located on the same node, optimizing query performance.
- **Clustering Column:** `itemInSession` - This clustering column ensures uniqueness and provides natural ordering by song sequence within the partition. When multiple rows share the same partition key, Cassandra automatically sorts them by the clustering column, which aligns with the requirement to sort songs by `itemInSession`.

**Why This Design:**
- The two-column partition key efficiently handles the query's filter criteria, grouping related data together.
- The clustering column sorts results by item in session without requiring a separate `ORDER BY` clause.

### Table 3: `song_users`

Purpose to retrieve all user names who listened to a specific song.

**Primary Key:**
- **Partition Key:** `song` - This is the partition key because Query 3 filters by `song = 'All Hands Against His Own'`. All users who listened to a specific song are stored in the same partition, enabling efficient retrieval.
- **Clustering Column:** `userId` - This clustering column ensures uniqueness (a user is listed only once per song in the partition) and provides efficient secondary sorting.

**Why This Design:**
- Query 3 filters exclusively on song name, making `song` the natural partition key.
- Using `userId` as a clustering column prevents duplicate entries for the same user-song combination and maintains data organization.

---


In [8]:
# Table 1: session_songs
"""
Creates a table to efficiently query songs by session ID and item position.

Partition Key: sessionId - Groups all songs for a given session together.
Clustering Column: itemInSession - Orders songs within a session and ensures uniqueness.

This design optimizes Query 1: Retrieve artist, song, and length for sessionId = 338, itemInSession = 4.
"""

query = """
CREATE TABLE IF NOT EXISTS session_songs (
    sessionId int,         -- Partition Key: Filters songs by specific session
    itemInSession int,     -- Clustering Column: Orders songs within session, ensures uniqueness
    artist text,           -- Song artist name
    song text,             -- Song title
    length float,          -- Song duration in seconds
    PRIMARY KEY (sessionId, itemInSession)
);
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Table 2: user_session_songs
"""
Creates a table to retrieve songs by user and session, sorted by item position.

Partition Key: (userId, sessionId) - Composite key groups all songs for a specific user-session pair.
Clustering Column: itemInSession - Orders songs within the partition and ensures uniqueness.

This design optimizes Query 2: Retrieve artist, song, and user name for userId = 10, sessionId = 182, sorted by itemInSession.
"""

query1 = """
CREATE TABLE IF NOT EXISTS user_session_songs (
    userId int,            -- Partition Key (part 1): User identifier
    sessionId int,         -- Partition Key (part 2): Session identifier
    itemInSession int,     -- Clustering Column: Orders songs in session sequence
    userName text,         -- User's first and last name
    artist text,           -- Song artist name
    song text,             -- Song title
    PRIMARY KEY((userId, sessionId), itemInSession)
);
"""
try:
    session.execute(query1)
except Exception as e:
    print(e)

# Table 3: song_users
"""
Creates a table to retrieve all users who listened to a specific song.

Partition Key: song - Groups all users for a specific song together.
Clustering Column: userId - Ensures uniqueness and provides secondary sorting.

This design optimizes Query 3: Retrieve all user names who listened to 'All Hands Against His Own'.
"""

query2 = """
CREATE TABLE IF NOT EXISTS song_users (
    song text,             -- Partition Key: Filters users by song title
    userId int,            -- Clustering Column: User identifier, ensures uniqueness
    userName text,         -- User's first and last name
    PRIMARY KEY (song, userId)
);
"""
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [9]:
# Insert into session_songs
# Load CSV using pandas
file = "event_datafile_new.csv"
df = pd.read_csv(file)

# Prepare CQL insert statement
query = """
INSERT INTO session_songs (
    sessionId,
    itemInSession,
    artist,
    song,
    length
) VALUES (%s, %s, %s, %s, %s)
"""

# Execute inserts
for _, row in df.iterrows():
    session.execute(
        query,
        (
            int(row["sessionId"]),
            int(row["itemInSession"]),
            row["artist"],
            row["song"],
            float(row["length"]),
        )
    )

In [12]:
# Insert into user_session_songs

# Load CSV using pandas
file = "event_datafile_new.csv"
df = pd.read_csv(file)

# CQL insert statement
query = """
INSERT INTO user_session_songs (
    userId,
    sessionId,
    itemInSession,
    userName,
    artist,
    song
) VALUES (%s, %s, %s, %s, %s, %s)
"""

# Execute inserts
for _, row in df.iterrows():
    session.execute(
        query,
        (
            int(row["userId"]),
            int(row["sessionId"]),
            int(row["itemInSession"]),
            f"{row['firstName']} {row['lastName']}",
            row["artist"],
            row["song"],
        )
    )

In [13]:
# Insert into song_users

# Load CSV using pandas
file = "event_datafile_new.csv"
df = pd.read_csv(file)

# CQL insert statement
query = """
INSERT INTO song_users (
    song,
    userId,
    userName
) VALUES (%s, %s, %s)
"""

# Execute inserts
for _, row in df.iterrows():
    session.execute(
        query,
        (
            row["song"],
            int(row["userId"]),
            f"{row['firstName']} {row['lastName']}",
        )
    )

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = """
 SELECT artist, song, length FROM session_songs
 WHERE sessionId = 338 AND itemInSession = 4;
"""
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}")


Artist: Faithless, Song: Music Matters (Mark Knight Dub)


In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """
 SELECT artist, song, userName FROM user_session_songs
 WHERE userId = 10 AND sessionId = 182;
"""
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, User name: {row.username}")


Artist: Down To The Bone, Song: Keep On Keepin' On, User name: Sylvie Cruz
Artist: Three Drives, Song: Greece 2000, User name: Sylvie Cruz
Artist: Sebastien Tellier, Song: Kilometer, User name: Sylvie Cruz
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User name: Sylvie Cruz


In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """
 SELECT userName FROM song_users
 WHERE song = 'All Hands Against His Own';
"""
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(f" {row.username}")


 Jacqueline Lynch
 Tegan Levine
 Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS session_songs")
except Exception as e:
    print(f"Error dropping session_songs: {e}")

try:
    session.execute("DROP TABLE IF EXISTS user_session_songs")
except Exception as e:
    print(f"Error dropping user_session_songs: {e}")

try:
    session.execute("DROP TABLE IF EXISTS song_users")
except Exception as e:
    print(f"Error dropping song_users: {e}")


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()